In [7]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torchvision.transforms as T


plt.rcParams["savefig.bbox"] = "tight"
# sphinx_gallery_thumbnail_number = 2


def plot(imgs, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0])
    _, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            img = F.to_pil_image(img.to("cpu"))
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    plt.tight_layout()

In [8]:
import tempfile
from pathlib import Path
from urllib.request import urlretrieve


# video_url = "https://download.pytorch.org/tutorial/pexelscom_pavel_danilyuk_basketball_hd.mp4"
video_path = "/content/vid1.mp4"

In [3]:
!pip install av


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 33.6 MB/s eta 0:00:00


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def preprocess(batch):
    transforms = T.Compose(
        [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
            T.Resize(size=(520, 960)),
        ]
    )
    batch = transforms(batch)
    return batch


# If you can, run this example on a GPU, it will be a lot faster.
device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
def writeFlow(name, flow):
    f = open(name, 'wb')
    f.write('PIEH'.encode('utf-8'))
    np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
    flow = flow.astype(np.float32)
    flow.tofile(f)

In [22]:
def readFlow(name):
    #if name.endswith('.pfm') or name.endswith('.PFM'):
        #return readPFM(name)[0][:,:,0:2]

    f = open(name, 'rb')

    header = f.read(4)
    if header.decode("utf-8") != 'PIEH':
        raise Exception('Flow file header does not contain PIEH')

    width = np.fromfile(f, np.int32, 1).squeeze()
    height = np.fromfile(f, np.int32, 1).squeeze()

    flow = np.fromfile(f, np.float32, width * height * 2).reshape((height, width, 2))

    return flow.astype(np.float32)

In [49]:
from scipy import misc
def writeImage(name, data):
    #if name.endswith('.pfm') or name.endswith('.PFM'):
        #return writePFM(name, data, 1)

    return misc.imsave(name, data)
from PIL import Image

In [52]:
from torchvision.io import read_video
from torchvision.models.optical_flow import raft_small

frames, _, _ = read_video(str(video_path))
frames = frames.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
ctr = 0

for j in range(24):
  video_path = f"/content/drive/MyDrive/Expt5/training_videos/vid{j+1}.mp4"
  for i in range(149):
    ctr = ctr+1
    frame_list1 = [frames[i]]
    frame_list2 = [frames[i+1]]
    img1_batch = torch.stack(frame_list1)
    img2_batch = torch.stack(frame_list2)
    img1_batch = preprocess(img1_batch).to(device)
    img2_batch = preprocess(img2_batch).to(device)
    model = raft_small(pretrained=True, progress=False).to(device)
    model = model.eval()

    list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
    tensor = list_of_flows[11][0]
    tensor = tensor.permute(1,2,0)
    file_path = f"/content/drive/MyDrive/Expt5/train/{ctr}_flow.flo"
    writeFlow(file_path,tensor.detach().numpy())



In [67]:
import cv2

ctr = 0
for j in range(24):
  video_path = f"/content/drive/MyDrive/Expt5/training_videos/vid{j+1}.mp4"
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
        print("Error: Could not open video file.")

  for i in range(149):
    ctr = ctr+1
    ret, frame = cap.read()
    if not ret:
            print("Error")
    frame_path = f"/content/drive/MyDrive/Expt5/train_img1/{ctr}_img1.ppm"
    if not cv2.imwrite(frame_path, frame):
      print(f"Error: {i} frame in {j+1} video")

  cap.release()

In [66]:
import cv2

ctr = 0
for j in range(24):
  video_path = f"/content/drive/MyDrive/Expt5/training_videos/vid{j+1}.mp4"
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
        print("Error: Could not open video file.")

  for i in range(1,150):
    ctr = ctr+1
    ret, frame = cap.read()
    if not ret:
            print("Error")
    frame_path = f"/content/drive/MyDrive/Expt5/train/{ctr}_img2.ppm"
    if not cv2.imwrite(frame_path, frame):
      print(f"Error: {i} frame in {j+1} video")

  cap.release()

In [ ]:
from torchvision.io import read_video
from torchvision.models.optical_flow import raft_small

frames, _, _ = read_video(str(video_path))
frames = frames.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
ctr = 0

for j in range(6):
  video_path = f"/content/drive/MyDrive/Expt5/testing_videos/vid{j+1}.mp4"
  for i in range(149):
    ctr = ctr+1
    frame_list1 = [frames[i]]
    frame_list2 = [frames[i+1]]
    img1_batch = torch.stack(frame_list1)
    img2_batch = torch.stack(frame_list2)
    img1_batch = preprocess(img1_batch).to(device)
    img2_batch = preprocess(img2_batch).to(device)
    model = raft_small(pretrained=True, progress=False).to(device)
    model = model.eval()

    list_of_flows = model(img1_batch.to(device), img2_batch.to(device))
    tensor = list_of_flows[11][0]
    tensor = tensor.permute(1,2,0)
    file_path = f"/content/drive/MyDrive/Expt5/test/{ctr}_flow.flo"
    writeFlow(file_path,tensor.detach().numpy())

In [ ]:
import cv2

ctr = 0
for j in range(24):
  video_path = f"/content/drive/MyDrive/Expt5/testing_videos/vid{j+1}.mp4"
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
        print("Error: Could not open video file.")

  for i in range(149):
    ctr = ctr+1
    ret, frame = cap.read()
    if not ret:
            print("Error")
    frame_path = f"/content/drive/MyDrive/Expt5/test/{ctr}_img1.ppm"
    if not cv2.imwrite(frame_path, frame):
      print(f"Error: {i} frame in {j+1} video")

  cap.release()

In [ ]:
import cv2

ctr = 0
for j in range(24):
  video_path = f"/content/drive/MyDrive/Expt5/training_videos/vid{j+1}.mp4"
  cap = cv2.VideoCapture(video_path)

  if not cap.isOpened():
        print("Error: Could not open video file.")

  for i in range(1,150):
    ctr = ctr+1
    ret, frame = cap.read()
    if not ret:
            print("Error")
    frame_path = f"/content/drive/MyDrive/Expt5/test/{ctr}_img2.ppm"
    if not cv2.imwrite(frame_path, frame):
      print(f"Error: {i} frame in {j+1} video")

  cap.release()

In [20]:
print(list_of_flows[11].shape)

torch.Size([1, 2, 520, 960])


In [42]:
file_name = '22862_flow.flo'
flow_array = readFlow(file_name)
print(flow_array.shape)
print(flow_array)

(384, 512, 2)
[[[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]]

 [[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]]

 [[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]]

 ...

 [[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]]

 [[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]]

 [[-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  ...
  [-16.585667  12.283061]
  [-16.585667  12.283061]
  [-16.585667  12.283061

In [38]:
tensor = list_of_flows[11][0]
print(tensor.shape)
tensor = tensor.permute(1,2,0)
print(tensor.shape)


torch.Size([2, 520, 960])
torch.Size([520, 960, 2])


In [41]:
writeFlow('flow.flo',tensor.detach().numpy())